In [ ]:
from pydantic import BaseModel, PrivateAttr
from threading import RLock
from typing import Callable

class SharedLock:
    lock = RLock()

class Shared(BaseModel):
    """
    This class provides in-memory shared data so that actions can communicate with each other during
    an api server's lifetime. Not meant to be persisted. Persistent shared data is a different feature.

    Usage:
        sh = Shareds.get('foo')
        def update(d:dict):
            d['a'] = 1
            return 1
        result = sh.apply(update)
        print(result, sh.copy_data())
    """
    # not treated as a model attr
    _data:dict = PrivateAttr(default_factory = dict)

    def copy_data(self):
        return self._data.copy()
    
    def apply(self, callable):
        """
        Transactionally applies callable to a copy of the data. A failure within
        the callable won't corrupt the dictionary.
        """
        with SharedLock.lock:
            copy = self._data.copy()
            result = callable(copy)
            self._data = copy
            return result

class Shareds:
    """
    A dictionary of instances of Shared.
    """
    singletons = {}

    @classmethod
    def get(cls, label:str) -> Shared:
        if label not in cls.singletons:
            cls.singletons[label] = Shared()
        return cls.singletons[label]
    
    @classmethod
    def key_set(cls):
        return set(cls.singletons.keys())



In [ ]:
s = Shareds.get('foo')
def update(d:dict):
    d['a'] = 1
    return 1
result = s.apply(update)
print(result, s.copy_data())

In [ ]:
from whendo.core.action import Action
from typing import List
from enum import Enum

class ListOpMode(Enum):
    ALL = 'all'
    OR = 'or'
    AND = 'and'

class ListAction(Action)
    op_mode:ListOpMode
    action_list:List[Action]:[]

    def execute(self, tag:str=None, scheduler_info:dict=None):
        processing_count, success_count, failure_count, successful_actions, exception_actions = process_action_list(
            op_mode=self.op_mode,
            action_list=self.action_list
            )
        if success_count == 0:
            return Exception(f"no action succeeded")
        else:
            processing_info = {
                'processing_count':processing_count,
                'success_count':success_count,
                'exception_count':failure_count,
                'successful_actions':successful_actions,
                'exception_actions':exception_actions
            }
            return {'outcome':'list action executed', 'action':self.info(), 'processing_info':processing_info}

class IfElseAction(Action):
    op_mode:ListOpMode
    test_action:Action
    if_actions:List[Action]
    else_action:Action

    def execute(self, tag:str=None, scheduler_info:dict=None):
        test_result = self.test_action.execute(tag=tag, scheduler_info=scheduler_info)
        processing_count = 1
        success_count = 0
        exception_count = 0
        successful_actions = []
        exception_actions = []
        else_test = isinstance(test_result, Exception)
        if else_test: # execute the else action
            exception_count = 1
            exception_actions.append(self.test_action.dict())

            else_result = else_action.execute(tag=tag, scheduler_info=scheduler_info)
            processing_count += 1
            if isinstance(else_result, Exception):
                exception_count += 1
                exception_actions.append(self.else_action.dict())
            else:
                success_count += 1
                successful_actions.append(self.else_action.dict())
        else:
            success_count += 1
            successful_actions.append(self.test_action.dict())
            processing_count, success_count, failure_count, successful_actions, exception_actions = process_action_list(
                op_mode=self.op_mode,
                action_list=self.if_actions,
                processing_count=processing_count,
                success_count=success_count,
                exception_count=exception_count,
                successful_actions=successful_actions,
                exception_actions=exception_actions
            )
        if success_count == 0:
            return Exception(f"no action succeeded")
        else:
            processing_info = {
                'else_test':else_test,
                'processing_count':processing_count,
                'success_count':success_count,
                'exception_count':failure_count,
                'successful_actions':successful_actions,
                'exception_actions':exception_actions
            }
            return {'outcome':'if-else action executed', 'action':self.info(), 'processing_info':processing_info}

def process_action_list(
    op_mode:ListOpMode,
    action_list:List[Action],
    processing_count:int=0,
    success_count:int=0,
    exception_count:int=0,
    successful_actions:List[Action]=[],
    exception_actions:List[Action]=[]
    ):
    processing_count = processing_count
    success_count = success_count
    exception_count = exception_count
    successful_actions = successful_actions
    exception_actions = exception_actions
    if op_mode == ListOpMode.ALL: # invoke all regardless of outcomes
        for action in action_list:
            result = action.execute(tag=tag, scheduler_info=scheduler_info)
            processing_count += 1
            if isinstance(result, Exception):
                exception_count += 1
                exception_actions.append(action.dict())
            else:
                success_count += 1
                successful_actions.append(action.dict())
    elif op_mode == ListOpMode.OR: # stop after first success
        for action in action_list:
            result = action.execute(tag=tag, scheduler_info=scheduler_info)
            processing_count += 1
            if isinstance(result, Exception):
                exception_count += 1
                exception_actions.append(action.dict())
            else:
                success_count += 1
                successful_actions.append(action.dict())
                break
    elif op_mode == ListOpMode.AND: # stop after first failure
        for action in action_list:
            action.execute(tag=tag, scheduler_info=scheduler_info)
            processing_count += 1
            if isinstance(result, Exception):
                exception_count += 1
                exception_actions.append(action.dict())
                break
            else:
                success_count += 1
                successful_actions.append(action.dict())
    return processing_count, success_count, failure_count, successful_actions, exception_actions
    }



### architecture thoughts

- 100% container based deployment
    - multiple containers per pi, distinguished by port
        - this could be a way of restoring previous behavior at a pi
    - container synonymous with mothership
- some machines (not necessarily pi's) could host multiple motherships 
    - these could be repositories for libraries of motherships, to be used as templates for pi's in the field
- a fleet is a collection of motherships
    - perhaps the fleet api would listen on 5050
        - the others listening on some other port
    - maybe an associated restful api that manages the fleet (a collection of mothership hosts)
    - there is a mothership sdk. there will be a fleet sdk.
    - actions would include
        - use a mothership's configuration to replace the configurations in other motherships
    - there would be a mothership discovery function
        - this would allow any machine to take of the role of fleet manager (taking fleet api requests)
        - plus a mothership health monitoring function


In [ ]:
import json
e = ['ack!', 'kack', 'whack']
isinstance(e, Exception)

